In [13]:
import json
import pandas as pd

In [14]:
heart = pd.read_json('~/Desktop/py129/self-viz/sample_heart.json')

# Empty lists for heartbeats/min and Fitbit 'Confidence' value
bpm_list = []
conf_list = []

# Populates empty lists with respective Fitbit values
for i in heart['value']:
    bpm_list.append(i['bpm'])
    conf_list.append(i['confidence'])

# Creates new pandas dataframes from populated lists
bpm_df = pd.DataFrame(bpm_list)
conf_df = pd.DataFrame(conf_list)

# Assigns dataframes as new series/columns in original dataframe
heart['BPM'] = bpm_df
heart['Confidence'] = conf_df

# Exports dataframe as CSV
heart[['dateTime','BPM','Confidence']].to_csv(('~/Desktop/py129/self-viz/sample_heart.csv'))


In [15]:
# Reads JSON file and renames column for Calories burned per minute
calor = pd.read_json('~/Desktop/py129/self-viz/sample_calories.json')
calor.rename(columns={'value':'Calories'}, inplace=True)

# Exports dataframe as CSV
calor.to_csv(('~/Desktop/py129/self-viz/sample_calories.csv'))


In [16]:
heart = pd.read_csv('~/Desktop/py129/self-viz/denver_trip.csv')

In [18]:
import psycopg2

try:
    # Creates connection to new DB, will create DB if nonexistant
    #conn = sqlite3.connect('heartcaltrip')
    conn = psycopg2.connect(
        database='heartcaltrip',
        user='steve_wortmann',
        password='paigek#624',
        host='localhost'
    )
    
    # Cursor will give us result set
    cursor = conn.cursor()
    print("Connected")
    
    hearttable_sql='''
                    CREATE TABLE IF NOT EXISTS
                        heart(
                            time_id INTEGER PRIMARY KEY,
                            dateTime TIMESTAMP,
                            Bpm INTEGER,
                            Confidence INTEGER
                        );
                '''
    cursor.execute(hearttable_sql)
    conn.commit()
    print("Created HEART Table")
    
    calorietable_sql='''
                    CREATE TABLE IF NOT EXISTS
                        calorie(
                            time_id INTEGER PRIMARY KEY,
                            dateTime TIMESTAMP,
                            Calories DOUBLE
                        );
                '''
    cursor.execute(calorietable_sql)
    conn.commit()
    print("Created CALORIE Table")

    triptable_sql='''
                    CREATE TABLE IF NOT EXISTS
                        trip(
                            time_id INTEGER PRIMARY KEY,
                            name TEXT,
                            address TEXT,
                            description TEXT,
                            TimeSpan_begin TIMESTAMP,
                            TimeSpan_end TIMESTAMP,
                            longitude DOUBLE,
                            latitude DOUBLE,
                            LineString_coordinates TEXT
                        );
                '''
    cursor.execute(triptable_sql)
    conn.commit()
    print("Created TRIP Table")
    
except psycopg2.Error as error:
        print("Error")
        print(error)
# Always close up our resources
finally:
    if cursor:
        cursor.close()
        if conn:
            conn.close()

Connected
Created HEART Table
Created CALORIE Table
Created TRIP Table


In [19]:
def close_DB_resources(dbconn, cursor):
    try:
        if dbconn:
            dbconn.close()
        if cursor:
            cursor.close()
        print("closed resources")
    except psycopg2.Error as err:
        ("Error closing resources")

In [20]:
# Utility for handling DB errors
def handle_DB_error(dbconn, cursor):
    if dbconn:
        try:
            dbconn.rollback()
            print("Rolled back transation")
        except psycopg2.Error as error:
            print("Error rolling back transaction")
        finally:
            close_DB_resources(dbconn, cursor)
            dbconn = None
            return dbconn

In [21]:
# Utility DB connection and cursor function to return our access objects
def connect_SQLite():
    try:
        dbconn = psycopg2.connect(
            database='heartcaltrip',
            user='steve_wortmann',
            password='paigek#624',
            host='localhost'
        )
        cursor = dbconn.cursor()
        print("Connected to heartcaltrip")
    except psycopg2.Error as error:
        print("Error opening database")
        dbconn = handle_DB_error(dbconn, cursor)
    return dbconn, cursor

In [22]:
import csv

heart_insert = '''INSERT INTO heart (time_id, dateTime, Bpm, Confidence)
                  VALUES (%s,%s,%s,%s);
                '''
# Retrieve my connection and cursor for use in all my inserts
dbconn, cursor = connect_SQLite()
with open('sample_heart.csv') as file:
    time_stamps = csv.DictReader(file)
    value_id = 0
    for time in time_stamps:
        value_id += 1
        record = (value_id, time['dateTime'], time['BPM'], time['Confidence'])
        print(record)
        cursor.execute(heart_insert, record)
        dbconn.commit()

close_DB_resources(dbconn, cursor)

Connected to heartcaltrip
(1, '2020-10-04 06:00:01', '130', '1')


InvalidTextRepresentation: invalid input syntax for type integer: "2020-10-04 06:00:01"
LINE 2:                   VALUES (1,'2020-10-04 06:00:01','130','1')...
                                    ^


In [10]:
calorie_insert = '''INSERT INTO calorie (time_id, dateTime, Calories)
                    VALUES (%s,%s,%s);
                '''
# Retrieve my connection and cursor for use in all my inserts
dbconn, cursor = connect_SQLite()
with open('sample_calories.csv') as file:
    time_stamps = csv.DictReader(file)
    value_id = 0
    for time in time_stamps:
        value_id += 1
        record = (value_id, time['dateTime'], time['Calories'])
        print(record)
        cursor.execute(calorie_insert, record)
        dbconn.commit()
        
close_DB_resources(dbconn, cursor)

Connected to heartcaltrip
(1, '2020-10-04 06:00:00', '4.72')


InvalidTextRepresentation: invalid input syntax for type integer: "4.72"
LINE 2:                     VALUES (1,'2020-10-04 06:00:00','4.72');
                                                            ^


In [11]:
trip_insert = '''INSERT INTO trip (time_id, name, address, description, TimeSpan_begin, TimeSpan_end, longitude, latitude, LineString_coordinates)
                 VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s);
                '''
# Retrieve my connection and cursor for use in all my inserts
dbconn, cursor = connect_SQLite()
with open('denver_trip.csv') as file:
    time_stamps = csv.DictReader(file)
    value_id = 0
    for time in time_stamps:
        value_id += 1
        record = (value_id, time['\ufeffname'], time['address'], time['description'], time['TimeSpan_begin'], time['TimeSpan_end'], time['longitude'], time['latitude'], time['LineString_coordinates'])
        print(record)
        cursor.execute(trip_insert, record)
        dbconn.commit()
        
close_DB_resources(dbconn, cursor)

Connected to heartcaltrip
(1, 'Home (701 Roosevelt Blvd)', '701 Roosevelt Blvd, Freedom, PA 15042', 'Building from 2020-10-03T00:21:12.327Z to 2020-10-03T08:22:40.539Z. Distance 0m', '2020-10-03T00:21:12.327Z', '2020-10-03T08:22:40.539Z', '-80.1614584', '40.6882963', '')


InvalidTextRepresentation: invalid input syntax for type integer: "-80.1614584"
LINE 2: ...0-10-03T00:21:12.327Z','2020-10-03T08:22:40.539Z','-80.16145...
                                                             ^


In [12]:
# Verify that we've got data

select_sql = '''
                SELECT *
                    FROM trip
                    WHERE address = '1441 Little Raven St, Denver, CO 80202';
            '''

dbconn, cursor = connect_SQLite()
cursor.execute(select_sql)
rec = cursor.fetchall()
for r in rec:
    print(r)

close_DB_resources(dbconn, cursor)

Connected to heartcaltrip
closed resources


# Script looks good, but can't overwrite 'heartcaltrip' because of postgres sessions...